In [1]:
#Mount Drive and install dependencies if running in Colab
def install_dependecies():
  !pip install transformers
  !pip install pytorch-lightning

from sys import path
import os
import sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive

  root_PATH = '/content/drive/My Drive/nlp-seminar/repository'
  drive_mount_location = '/content/drive'
  module_path = root_PATH + '/src'
  
  drive.mount(drive_mount_location, force_remount=True)
  path.append(root_PATH)

  install_dependecies()
else:
  root_PATH = os.path.abspath("../../..")
  module_path = os.path.abspath(os.path.join('../../../src'))

%load_ext autoreload
%autoreload 2

if module_path not in sys.path:
    sys.path.append(module_path)

Mounted at /content/drive
     |████████████████████████████████| 1.5MB 5.2MB/s 
     |████████████████████████████████| 2.9MB 18.2MB/s 
     |████████████████████████████████| 890kB 38.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8a5fd079f836cd6f2723901b75c49343552cd79b83eb44f99a26499ec0e1095b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 675kB 5.9MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 276kB 35.6MB/s 
     |████████████████████████████████| 829kB 19.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=efe890ece6ff9226e6e60522ce3446f71fa8edce004704c5de632d99ee62f509
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel fo

In [ ]:
from data_processor import DataProcessor
from model_evaluator import ModelEvaluator
from custom_dataset import CustomDataset
from models.roberta_custom_model import RoBERTaCustomModel

import pandas as pd
from torch import cuda
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import transformers
import numpy as np
from sklearn import metrics
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import accuracy_score
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [2]:
# Script configuration
MAX_LEN = 200
TEST_BATCH_SIZE = 32

model_to_use = 'roberta-base'
model_path = root_PATH + '/content/drive/MyDrive/nlp-seminar/repository/models/RoBERTa/tb_logs/my_RoBERTa_model/version_0/checkpoints/epoch=5-step=165.ckpt'

#Model parameters
model_params = {
                'test_batch_size': TEST_BATCH_SIZE
                }

gpus_to_use = [0]

In [ ]:
#Load datasets
test_df = pd.read_csv(root_PATH + '/data/test.csv')

#Get boolean mask of the datasets
test_boolean_mask = test_df.iloc[:,8:]

#Get topics present in the dataset
remaining_topics = test_boolean_mask.columns.tolist()

#Create new column with the boolean mask
test_df["list"] = test_boolean_mask.values.tolist()

#Remove columns not necessary from the datasets
test_df = test_df[["conversation","list"]].reset_index()

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(model_to_use)

testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)
data_loader = DataLoader(testing_set, batch_size=TEST_BATCH_SIZE, shuffle= False)

In [ ]:
model = RoBERTaCustomModel.load_from_checkpoint(
    model_path, 
    hparams = model_params, 
    training_dataset=None, 
    validation_dataset=None, 
    labels=remaining_topics, 
    model_to_use=model_to_use
    )

trainer = pl.Trainer(gpus=gpus_to_use,
                             #accelerator='dp',
                             #limit_train_batches=32,
                             #limit_val_batches=32,
                             #limit_test_batches=60,
                             #max_epochs=200,
                             #logger=logger,
                             #default_root_dir=logs_path
                             )

trainer.test(ckpt_path=model_path,model=model,test_dataloaders=[data_loader])